In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# ── 검색 키워드 설정
keyword = "파이썬"
search_url = f"https://www.jobkorea.co.kr/Search/?stext={urllib.parse.quote(keyword)}"

# ── 요청 헤더 설정 (크롤링 차단 우회용)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

# ── HTML 가져오기
resp = requests.get(search_url, headers=HEADERS)
soup = BeautifulSoup(resp.text, "html.parser")

# ── 공고 제목 추출 (구조 기반 접근)
titles = []

# a 태그 중 href가 "/Recruit/GI_Read/"로 시작하는 것만
for a in soup.find_all("a", href=True):
    if a["href"].startswith("/Recruit/GI_Read/"):
        title = a.get_text(strip=True)
        if title and len(title) > 5:
            titles.append(title)
        if len(titles) >= 5:
            break

# ── 출력
print("🔍 잡코리아 채용 공고 (상위 5개):")
for i, t in enumerate(titles, 1):
    print(f"{i}. {t}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# ── 검색 키워드 설정
keyword = "파이썬"
search_url = f"https://www.jobkorea.co.kr/Search/?stext={urllib.parse.quote(keyword)}"

# ── 요청 헤더 설정 (크롤링 차단 우회용)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

# ── 1) HTML 가져오기 & 상태 확인
resp = requests.get(search_url, headers=HEADERS, timeout=10)
print("🔍 HTTP 상태 코드:", resp.status_code)
if resp.status_code != 200:
    print("! ERROR: 페이지를 불러오지 못했습니다.")
    exit(1)

html = resp.text
print("🔍 HTML 처음 500자:\n", html[:500].replace("\n", " "), "...\n")

# ── 2) BeautifulSoup 파싱 후 전체 <a> 태그 개수 확인
soup = BeautifulSoup(html, "html.parser")
all_a = soup.find_all("a", href=True)
print(f"🔍 전체 <a> 태그 개수: {len(all_a)}")
print("🔍 처음 20개 href/text 샘플:")
for i, a in enumerate(all_a[:20], 1):
    print(f"  {i:2d}. href={a['href']!r}, text={a.get_text(strip=True)!r}")
print()

# ── 3) 필터링된 공고 제목 추출
titles = []
for a in all_a:
    href = a["href"]
    if href.startswith("/Recruit/GI_Read/"):
        print("✔️ 매칭된 href:", href)  # 필터 조건에 걸릴 때마다 출력
        title = a.get_text(strip=True)
        print("   text:", title)
        if title and len(title) > 5:
            titles.append(title)
        if len(titles) >= 5:
            break

# ── 4) 결과 출력
if titles:
    print("\n🔍 잡코리아 채용 공고 (상위 5개):")
    for i, t in enumerate(titles, 1):
        print(f"{i}. {t}")
else:
    print("\n! WARNING: 필터링된 공고를 하나도 찾지 못했습니다.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# ── 검색 키워드 설정
keyword = "파이썬"
base_url = "https://www.jobkorea.co.kr"
search_url = f"{base_url}/Search/?stext={urllib.parse.quote(keyword)}"

# ── 요청 헤더 설정 (크롤링 차단 우회용)
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

# ── 1) HTML 가져오기
resp = requests.get(search_url, headers=HEADERS, timeout=10)
print("🔍 HTTP 상태 코드:", resp.status_code)
html = resp.text

# ── 2) BeautifulSoup 파싱
soup = BeautifulSoup(html, "html.parser")
all_a = soup.find_all("a", href=True)
print(f"🔍 전체 <a> 태그 개수: {len(all_a)}")

# ── 3) 공고 링크/제목 수집
titles = []
seen_urls = set()

for a in all_a:
    href = a["href"]
    # "/Recruit/GI_Read/"가 포함된 링크라면 절대/상대 모두 잡기
    if "/Recruit/GI_Read/" in href:
        # 절대경로로 변환
        full_url = urllib.parse.urljoin(base_url, href)
        if full_url in seen_urls:
            continue
        seen_urls.add(full_url)

        title = a.get_text(strip=True)
        print(f"✔️ 발견: URL={full_url}, text={title!r}")

        # 원하는 최소 길이만 남기고
        if title and len(title) > 10:
            titles.append((title, full_url))

        if len(titles) >= 5:
            break

# ── 4) 결과 출력
if titles:
    print("\n🔍 잡코리아 채용 공고 (상위 5개):")
    for i, (t, u) in enumerate(titles, 1):
        print(f"{i}. {t}\n   {u}\n")
else:
    print("\n! WARNING: 필터링된 공고를 하나도 찾지 못했습니다.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# ── 사용자 입력
keyword = input('키워드를 입력하세요: ')
page_num = int(input('크롤링할 페이지 수를 입력하세요: '))  # 숫자로 바로 변환
base_url = "https://www.jobkorea.co.kr"

# ── 요청 헤더
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}

# ── 결과를 저장할 리스트/집합은 루프 밖에서 선언!
titles = []
seen = set()

for n in range(1, page_num + 1):           # 1부터 page_num까지
    search_url = (
        f"{base_url}/Search/?"
        f"stext={urllib.parse.quote(keyword)}&"
        f"tabType=recruit&Page_No={n}"
    )
    try:
        resp = requests.get(search_url, headers=HEADERS, timeout=10)
        # HTTP 요청을 보낸 뒤 응답 코드가 200번대(성공)이 아닌 경우에 즉시 예외를 발생시켜줌.
        # 만약 404인 에러가 터졌을경우 바로 아래에 있는 except블록에서 잡아서 처리함.
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"[페이지 {n}] 요청 실패:", e)
        continue

    # ── a 태그 중 채용공고 링크만 골라서
    for a in soup.select("a[href*='/Recruit/GI_Read/']"):
        href = a["href"]
        full_url = urllib.parse.urljoin(base_url, href)

        if full_url in seen:
            continue
        seen.add(full_url)

        # ── <a> 바로 아래 <span> 에 제목이 감싸여 있다면
        span = a.find("span")
        if not span:
            continue  # span이 없으면 다음 링크로

        title = span.get_text(strip=True)
        if len(title) < 5:
            continue

        titles.append((title, full_url))

# ── 출력
print(f"🔍 잡코리아 채용 공고 (총 {len(titles)}건):")
for i, (t, u) in enumerate(titles, 1):
    print(f"{i}. {t}\n   {u}\n")


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = (
    "https://www.jobkorea.co.kr/Recruit/GI_Read/"
    "47370797?Oem_Code=C1&logpath=1&"
    "stext=%ED%8C%8C%EC%9D%B4%EC%8D%AC&"
    "listno=2&sc=631"
)
headers = {"User-Agent": "Mozilla/5.0"}
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

# 1) 회사명 추출: 클래스명에 'coName'이 포함된 span
co_tag = soup.find("span", class_=re.compile(r"coName"))
company_name = co_tag.get_text(strip=True) if co_tag else "추출 실패"
print("회사명:", company_name)

# 2) 공고 제목 추출:
#    h3 태그 중 클래스에 'hd_'로 시작하는 것 찾기
h3 = soup.find("h3", class_=re.compile(r"hd_"))
job_title = None
if h3:
    # h3태그 내부에 있는 모든 span태그 제거
    for span in h3.find_all("span"):
        span.extract()
    # h3태그 내부에 있는 p태그중 class가 'txt'인애들만 제거
    for p_txt in h3.find_all("p", class_=re.compile(r"txt")):
        p_txt.extract()
    # 남은 텍스트가 제목
    job_title = h3.get_text(strip=True)
print("공고 제목:", job_title or "추출 실패")

# 4) 상세 지원자격(경력, 학력, 스킬) 추출
qual_details = {}
# tbRow clear 클래스 가진 div 전부 순회
for div in soup.find_all("div", class_=re.compile(r"tbRow.*clear")):
    h4 = div.find("h4")
    # h4 텍스트에 '지원자격'이 포함된 블록만 선택
    if h4 and "지원자격" in h4.get_text():
        dl = div.find("dl", class_="tbList")
        if dl:
            # dt/dd 쌍으로 순회하며 딕셔너리에 저장
            for dt, dd in zip(dl.find_all("dt"), dl.find_all("dd")):
                key = dt.get_text(strip=True)
                val = dd.get_text(strip=True)
                qual_details[key] = val
        break

qual = extract_section("지원자격")
work = extract_section("근무조건")
print("지원자격 상세:")
for k, v in qual_details.items():
    print(f" - {k}: {v}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import re

url = (
    "https://www.jobkorea.co.kr/Recruit/GI_Read/"
    "47370797?Oem_Code=C1&logpath=1&"
    "stext=%ED%8C%8C%EC%9D%B4%EC%8D%AC&"
    "listno=2&sc=631"
)
headers = {"User-Agent": "Mozilla/5.0"}
resp = requests.get(url, headers=headers)
print("🔍 HTTP 상태 코드:", resp.status_code)

html = resp.text
print("\n── HTML 처음 500자 ──")
print(html[:500].replace("\n", " "), "\n…\n")

soup = BeautifulSoup(html, "html.parser")

# 1) 모든 span 태그와 클래스 찍어보기
print("── ALL <span> TAGS & CLASSES ──")
for span in soup.find_all("span"):
    print("span:", repr(span.get_text(strip=True)), "| class:", span.get("class"))
print()

# 2) 'coName' 방식으로 찾는 span 태그 찍어보기
print("── FIND span[class*=coName] ──")
for span in soup.find_all("span", class_=re.compile(r"coName")):
    print(span)
print()

# 3) 모든 h3 태그와 클래스 찍어보기
print("── ALL <h3> TAGS & CLASSES ──")
for h3 in soup.find_all("h3"):
    print("h3 class:", h3.get("class"), "| html:", h3.prettify())
print()

# 4) 'hd_' 패턴으로 찾는 h3 태그 찍어보기
print("── FIND h3[class^=hd_] ──")
for h3 in soup.find_all("h3", class_=re.compile(r"hd_")):
    print(h3.prettify())
print()

# 5) tbRow clear 블록 찍어보기
print("── FIND div[class*=tbRow] ──")
for div in soup.find_all("div", class_=re.compile(r"tbRow")):
    h4 = div.find("h4")
    print("BLOCK h4 text:", h4.get_text(strip=True) if h4 else "<no h4>")
    print(div.prettify(), "\n---\n")


In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re

# ── 설정
HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/114.0.0.0 Safari/537.36"
    )
}
BASE_URL = "https://www.jobkorea.co.kr"

def get_job_links(keyword, page_num):
    links = []
    seen = set()
    for n in range(1, page_num + 1):
        search_url = (
            f"{BASE_URL}/Search/?"
            f"stext={urllib.parse.quote(keyword)}&"
            f"tabType=recruit&Page_No={n}"
        )
        try:
            resp = requests.get(search_url, headers=HEADERS, timeout=10)
            resp.raise_for_status()
        except Exception as e:
            print(f"[페이지 {n}] 요청 실패:", e)
            continue

        soup = BeautifulSoup(resp.text, "html.parser")
        for a in soup.select("a[href*='/Recruit/GI_Read/']"):
            href = a["href"]
            full_url = urllib.parse.urljoin(BASE_URL, href)
            if full_url in seen:
                continue
            seen.add(full_url)

            span = a.find("span")
            if not span:
                continue
            title = span.get_text(strip=True)
            if len(title) < 5:
                continue

            links.append((title, full_url))
    return links

def parse_job_detail(url):
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        print(f"[상세] {url} 요청 실패:", e)
        return None

    soup = BeautifulSoup(resp.text, "html.parser")

    co_tag = soup.find("span", class_=re.compile(r"coName"))
    company_name = co_tag.get_text(strip=True) if co_tag else ""

    h3 = soup.find("h3", class_=re.compile(r"hd_"))
    job_title = ""
    if h3:
        for span in h3.find_all("span"):
            span.extract()
        for p_txt in h3.find_all("p", class_=re.compile(r"txt")):
            p_txt.extract()
        job_title = h3.get_text(strip=True)

    def extract_section(title):
        tit = soup.find("div", string=re.compile(title))
        if tit:
            content = tit.find_next_sibling("div")
            return content.get_text(strip=True) if content else ""
        return ""
    simple_qual = extract_section("지원자격")
    work = extract_section("근무조건")

    qual_details = {}
    for div in soup.find_all("div", class_=re.compile(r"tbRow.*clear")):
        h4 = div.find("h4")
        if h4 and "지원자격" in h4.get_text():
            dl = div.find("dl", class_="tbList")
            if dl:
                for dt, dd in zip(dl.find_all("dt"), dl.find_all("dd")):
                    qual_details[dt.get_text(strip=True)] = dd.get_text(strip=True)
            break

    return {
        "company_name": company_name,
        "job_title": job_title,
        "simple_qual": simple_qual,
        "work": work,
        "qual_details": qual_details
    }

def main():
    keyword = input("키워드를 입력하세요: ")
    page_num = int(input("크롤링할 페이지 수를 입력하세요: "))
    links = get_job_links(keyword, page_num)

    with open("jobKorea.txt", "w", encoding="utf-8") as f:
        for idx, (_, url) in enumerate(links, 1):
            f.write(f"=== 공고 {idx} ===\n")

            data = parse_job_detail(url)
            if not data:
                f.write("상세 정보 파싱 실패\n\n")
                continue

            f.write(f"회사명: {data['company_name']}\n")
            f.write(f"공고 제목: {data['job_title']}\n")
            f.write(f"지원자격 (요약): {data['simple_qual']}\n")
            f.write(f"근무조건: {data['work']}\n")
            f.write("지원자격 상세:\n")
            for k, v in data["qual_details"].items():
                f.write(f"  - {k}: {v}\n")
            f.write("\n")

    print(f"완료: jobKorea.txt 파일에 {len(links)}건 저장했습니다.")

if __name__ == "__main__":
    main()